# 객담도말 결핵진단 딥러닝 모델

CNN 기반의 객담도말 결핵진단 딥러닝 모델 소스코드입니다.

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution1D, Convolution2D, MaxPooling2D
from keras.utils import np_utils



batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3
model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta')

model.fit_generator(myGenerator(), samples_per_epoch = 60000, nb_epoch = 2, verbose=2, show_accuracy=True, callbacks=[], validation_data=None, class_weight=None, nb_worker=1)


Using Theano backend.


NameError: name 'myGenerator' is not defined

In [1]:
import os

from __future__ import print_function
import numpy as np

np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using Theano backend.
Using gpu device 0: GeForce GTX 660 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5005)


딥러닝 모델관련 환경설정입니다.

In [2]:
BATCH_SIZE = 128 # 한 epoch에서 실행시키는 단위(배치)크기
NUM_CLASSES = 2 # 클래스 수
NUM_EPOCHS = 1 # epoch 수
NUM_FILTERS = 32 # convolution 필터 수
NUM_POOL = 2 # max plling을 위한 pooling 영역 크기
NUM_CONV = 3 # convolution 커널 크기

데이터셋 관련 환경설정입니다.

In [3]:
IMG_CHANNELS = 1
IMG_ROWS = 64
IMG_COLS = 64

TRAIN_DATA_COUNT = 447648

train_img_filename = './dataset/train_image_64x64_gray_447648.bin'
train_label_filename = './dataset/train_label_64x64_gray_447648.bin'

TEST_DATA_COUNT = 15873

test_img_filename = './dataset/test_image_64x64_gray_15873.bin'
test_label_filename = './dataset/test_label_64x64_gray_15873.bin'

VALIDATION_DATA_COUNT = int(TRAIN_DATA_COUNT * 1.0/4.0)

In [4]:
MODEL_SAVE_FILE_PATH = './seq_model_cnn.h5'
PREDICT_FILE_PATH = './predict.txt'

img 자료 로딩 함수 입니다.

In [5]:
def load_img(filename, count, channel, row, col):
    print('Loading data from', filename)

    print('file size : ', os.path.getsize(filename))
    print('calc size : ', count * channel * row * col)
    
    fp = open(filename, 'rb')
    buf = fp.read(count * channel * row * col)
    data = np.frombuffer(buf, dtype=np.uint8)
    data = data.reshape(count, channel, row, col)

    print('loaded shape : ', data.shape)

    data = data.astype('float32')
    data /= 255

    return data

label 자료 로딩함수입니다.

In [6]:
def load_label(filename, count, classes):
    print('Loading labels from ', filename)
    
    print('file size : ', os.path.getsize(filename))
    print('calc size : ', count)
    
    fp = open(filename, 'r')
    buf = fp.read(count)
        
    data_bin = []
    for i in buf:
        data_bin.append(i)
    data = np.asarray(data_bin, dtype=np.uint8, order='C')

    print('loaded shape : ', data.shape)
    
    label_hist = np.histogram(data, bins=range(NUM_CLASSES+1))
    print(label_hist)
    
    # convert class vectors to binary class matrices
    data = np_utils.to_categorical(data, classes)

    return data

In [7]:
# the data, shuffled and split between train and test sets
train_img = load_img(train_img_filename, TRAIN_DATA_COUNT, IMG_CHANNELS, IMG_ROWS, IMG_COLS)
test_img = load_img(test_img_filename, TEST_DATA_COUNT, IMG_CHANNELS, IMG_ROWS, IMG_COLS)
#validation_img = load_img(validation_img_filename, VALIDATION_DATA_COUNT, IMG_CHANNELS, IMG_ROWS, IMG_COLS)

train_label = load_label(train_label_filename, TRAIN_DATA_COUNT, NUM_CLASSES)
test_label = load_label(test_label_filename, TEST_DATA_COUNT, NUM_CLASSES)
#validation_label = load_label(validation_label_filename, VALIDATION_DATA_COUNT, NUM_CLASSES)

Loading data from ./dataset/train_image_64x64_gray_447648.bin
file size :  1833566208
calc size :  1833566208
loaded shape :  (447648, 1, 64, 64)
Loading data from ./dataset/test_image_64x64_gray_15873.bin
file size :  65015808
calc size :  65015808
loaded shape :  (15873, 1, 64, 64)
Loading labels from  ./dataset/train_label_64x64_gray_447648.bin
file size :  447648
calc size :  447648
loaded shape :  (447648,)
(array([354528,  93120]), array([0, 1, 2]))
Loading labels from  ./dataset/test_label_64x64_gray_15873.bin
file size :  15873
calc size :  15873
loaded shape :  (15873,)
(array([15520,   353]), array([0, 1, 2]))


훈련셋의 일부로부터 검증셋을 생성합니다.

In [8]:
validation_img = train_img[:VALIDATION_DATA_COUNT, ...]
validation_label = train_label[:VALIDATION_DATA_COUNT, ...]

train_img = train_img[VALIDATION_DATA_COUNT:, ...]
train_label = train_label[VALIDATION_DATA_COUNT:, ...]

print('train count : ' + str(len(train_img)))
print('validation count : ' + str(len(validation_img)))

train count : 335736
validation count : 111912


딥리닝 모델을 구축합니다.

In [9]:
model = Sequential()
 
model.add(Convolution2D(NUM_FILTERS, NUM_CONV, NUM_CONV,
                        border_mode='valid',
                        input_shape=(IMG_CHANNELS, IMG_ROWS, IMG_COLS)))
model.add(Activation('relu'))
model.add(Convolution2D(NUM_FILTERS, NUM_CONV, NUM_CONV))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(NUM_POOL, NUM_POOL)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

# np_utils.visualize_util.plot(model, to_file='model.png')

딥러닝 모델을 구축합니다.

In [1]:


def myGenerator():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    y_train = np_utils.to_categorical(y_train,10)
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    while 1:
        for i in range(1875): # 1875 * 32 = 60000 -> # of training samples
            print("[")
            if i%125==0:
                print "i = " + str(i)
            yield X_train[i*32:(i+1)*32], y_train[i*32:(i+1)*32]
            print("]")
        print("-----")

NameError: name 'callbacks' is not defined

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(train_img, 
          train_label, 
          batch_size=BATCH_SIZE, 
          nb_epoch=NUM_EPOCHS,
          verbose=1, 
          validation_data=(validation_img, validation_label))

Train on 335736 samples, validate on 111912 samples
Epoch 1/1
335736/335736 [==============================] - 310s - loss: 0.1833 - acc: 0.9235 - val_loss: 0.1511 - val_acc: 0.9360


딥러닝 모델 테스트를 수행합니다.

In [11]:
score = model.evaluate(test_img, test_label, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

15873/15873 [==============================] - 4s     
Test score: 0.0604064624439
Test accuracy: 0.97668997669


In [ ]:
classes = model.predict_classes(test_img, batch_size=32)
np.savetxt(PREDICT_FILE_PATH, classes, fmt='%d')

15873/15873 [==============================] - 4s     


In [ ]:
model.summary()
model.save_weights(MODEL_SAVE_FILE_PATH)

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 62, 62)  320         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 62, 62)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 60, 60)  9248        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)          (None, 32, 60, 60)  0           convolution2d_2[0][0]            
___________________________________________________________________________________________